In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import keras
import random
import numpy as np

In [57]:
text=open('datasets/CactusForestDrive.txt').read().lower()

text=text[1302:]
len(text)
text

'national park service, u. s. department of the interior. they\ninclude such outstanding scenic areas as grand canyon and yosemite\nnational parks and other parks and monuments set aside for their scenic,\nscientific, historic or archeologic values. these superb areas are yours\nand are a part of your heritage as american citizens.\n\nthe national park service has the job of preserving the parks and\nmonuments in their natural, unspoiled condition and of making them\navailable for your enjoyment in such a manner as to leave them\nunimpaired for the inspiration of future generations. to achieve this\nhigh purpose it has been necessary to prohibit such activities as\nwoodcutting, hunting, grazing, mining and even flower-picking. the men\nin the uniform of the national park service are here to serve you, and\nwill welcome the opportunity to make your stay at saguaro national\nmonument more enjoyable.\n\nwe hope you will cooperate with us “by taking only pictures and\ninspiration and leavi

In [4]:
for ch in ['0','1', '2', '3', '4', '5', '6', '7', '8', '9', '!', '"', '$', '%', '&', '~', '`', '(', ')', '*',
          '-', '/', ';', '@', '?', ':', '©', '¢', 'ã', '\xa0', '\n', '\r', '.']:
    if ch in text:
        text=text.replace(ch,' ')
print(set(text))

{'o', 'i', '“', 'e', 's', 'c', ' ', 'r', '_', ',', 'j', 'q', 'h', 'b', '—', '[', ']', 'z', 'd', 'a', '”', 't', 'f', 'v', 'u', '’', "'", 'k', 'p', 'm', 'g', 'w', 'n', 'x', 'y', 'l'}


In [58]:
chars=sorted(set(list(text)))
char_to_indices= dict((c,i) for i,c in enumerate(chars))
indices_to_char= dict((i,c) for i,c in enumerate(chars))
indices_to_char

{0: '\n',
 1: ' ',
 2: '!',
 3: '"',
 4: '$',
 5: '%',
 6: "'",
 7: '(',
 8: ')',
 9: '*',
 10: ',',
 11: '-',
 12: '.',
 13: '/',
 14: '0',
 15: '1',
 16: '2',
 17: '3',
 18: '4',
 19: '5',
 20: '6',
 21: '7',
 22: '8',
 23: '9',
 24: ':',
 25: ';',
 26: '?',
 27: '@',
 28: '[',
 29: ']',
 30: '_',
 31: 'a',
 32: 'b',
 33: 'c',
 34: 'd',
 35: 'e',
 36: 'f',
 37: 'g',
 38: 'h',
 39: 'i',
 40: 'j',
 41: 'k',
 42: 'l',
 43: 'm',
 44: 'n',
 45: 'o',
 46: 'p',
 47: 'q',
 48: 'r',
 49: 's',
 50: 't',
 51: 'u',
 52: 'v',
 53: 'w',
 54: 'x',
 55: 'y',
 56: 'z',
 57: '—',
 58: '’',
 59: '“',
 60: '”'}

In [18]:
def create_datasets(text,window,step):
    input,output=[],[]
    n_batches=int((len(text)-window)/step)
    for i in range(n_batches-1):
        input.append(text[i*step:i*step+window])
        output.append(text[i*step+window])
    return input,output

window=50
step=3
input,output=create_datasets(text,window,step)
input[11], output[11]

('ational monument is one of more than     units adm', 'i')

In [59]:
def encodeio(text,window,step):
    num_chars=len(chars)
    
    input,output=create_datasets(text,window,step)
    x=np.zeros((len(input),window,num_chars),dtype=bool)
    y=np.zeros((len(input),num_chars),dtype=bool)
    for i,sentence in enumerate(input):
        for j,char in enumerate(sentence):
            x[i,j,char_to_indices[char]]=1
        y[i,char_to_indices[output[i]]]=1
    return x,y


x,y=encodeio(text,window,step)
print(x.shape,y.shape)

(13748, 50, 61) (13748, 61)


In [75]:
model=Sequential([
    keras.layers.LSTM(120,input_shape=(window,len(chars))),
    keras.layers.Dropout(0.22),
    keras.layers.Dense(len(chars),activation='linear'),
    keras.layers.Dense(len(chars),activation='softmax')
])
xsmall=x[:20000,::]
ysmall=y[:20000,:]
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x,y,epochs=10,batch_size=500)

Epoch 1/10
28/28 [==============================] - 2s 16ms/step - loss: 3.5475 - accuracy: 0.1632
Epoch 2/10
28/28 [==============================] - 0s 9ms/step - loss: 3.0544 - accuracy: 0.1841
Epoch 3/10
28/28 [==============================] - 0s 9ms/step - loss: 3.0215 - accuracy: 0.1849
Epoch 4/10
28/28 [==============================] - 0s 9ms/step - loss: 2.9990 - accuracy: 0.1845
Epoch 5/10
28/28 [==============================] - 0s 9ms/step - loss: 2.9616 - accuracy: 0.1871
Epoch 6/10
28/28 [==============================] - 0s 9ms/step - loss: 2.9124 - accuracy: 0.1960
Epoch 7/10
28/28 [==============================] - 0s 9ms/step - loss: 2.8498 - accuracy: 0.2202
Epoch 8/10
28/28 [==============================] - 0s 9ms/step - loss: 2.7709 - accuracy: 0.2367
Epoch 9/10
28/28 [==============================] - 0s 9ms/step - loss: 2.6983 - accuracy: 0.2583
Epoch 10/10
28/28 [==============================] - 0s 9ms/step - loss: 2.6300 - accuracy: 0.2795


In [77]:
def predict_next_chars(model,input_chars,num_to_predict):
    pred=''
    for i in range(num_to_predict):
        x_test=np.zeros((1,window,len(chars)))

        for i, char in enumerate(input_chars):
            x_test[0,i,char_to_indices[char]]=1
        x_predict=model.predict(x_test,verbose=0)[0]
        
        r=np.argmax(x_predict)
        # print(r)
        d=indices_to_char[r]
        # print(d)
        pred+=d
        input_chars+=d
        input_chars=input_chars[1:]
    return pred

start=84
num_to_predict=10
input_chars = text[start:start+window]
print('complete sequence:', text[start:start+window+num_to_predict])
print('input sequence:', input_chars)
print('output sequence:', predict_next_chars(model,input_chars,num_to_predict=num_to_predict))

complete sequence: ng scenic areas as grand canyon and yosemite
national parks 
input sequence: ng scenic areas as grand canyon and yosemite
natio
output sequence: n  on  or 
